# Some preliminaries...

In [1]:
import glob
import sys
from collections import defaultdict

ext = ".txt"
inswild = "instances/*"+ext

# Loop over all .txt files and create .gmsdat files

In [2]:
for fname in glob.glob(inswild):
    
    instance = fname.replace(ext,"")
    
    watersources = []
    processunits = []
    treatmentunits = []
    buffers = []
    destinations = []
    contaminants = []
    times = []
    
    sourcetimecontconc = defaultdict(lambda: defaultdict(dict))
    sourcetimecost = defaultdict(lambda: defaultdict(dict))
    sourcetimeminflow = defaultdict(lambda: defaultdict(dict))
    sourcetimemaxflow = defaultdict(lambda: defaultdict(dict))
    
    PUminFlow = {}
    PUmaxFlow = {}
    PUbalance = {}
    PUstart = {}
    PUduration = {}
    PUend = {}
    PUcontdisc = defaultdict(lambda: defaultdict(dict))
    PUcontin = defaultdict(lambda: defaultdict(dict))
    PUcontout = defaultdict(lambda: defaultdict(dict))
    
    TUtype = {}
    TUinvCost = {}
    TUopCost = {}
    TUminFlow = {}
    TUmaxFlow = {}
    TUduration = {}
    TUconttypechar = defaultdict(lambda: defaultdict(dict))
    TUcontin = defaultdict(lambda: defaultdict(dict))
    
    desttimecontconc = defaultdict(lambda: defaultdict(dict))
    desttimeminflow = defaultdict(lambda: defaultdict(dict))
    desttimemaxflow = defaultdict(lambda: defaultdict(dict))
    
    f = open(fname, 'r')
    
    fout = open (instance+".gmsdat",'w')
    
    for line in f:
        
        if line.split('\t')[0].strip() == 'Water Sources':
            n = line.split('\t')[1].strip()
            watersources = line.split('\t')[2:int(n)+2]
            
        if line.split('\t')[0].strip() == 'Process Units':
            n = line.split('\t')[1].strip()
            processunits = line.split('\t')[2:int(n)+2]
            
        if line.split('\t')[0].strip() == 'Treatment Units':
            n = line.split('\t')[1].strip()
            treatmentunits = line.split('\t')[2:int(n)+2]
            
        if line.split('\t')[0].strip() == 'Buffer Units':
            n = line.split('\t')[1].strip()
            buffers = line.split('\t')[2:int(n)+2]
            
        if line.split('\t')[0].strip() == 'Water Destination':
            n = line.split('\t')[1].strip()
            destinations = line.split('\t')[2:int(n)+2]
            
        if line.split('\t')[0].strip() == 'Contaminants':
            n = line.split('\t')[1].strip()
            contaminants = line.split('\t')[2:int(n)+2]
            
        if line.split('\t')[0].strip() == 'Time Periods (Planning Horizon)':
            n = line.split('\t')[1].strip()
            times = line.split('\t')[2:int(n)+2]
        
        if line.split('\t')[0].strip() == 'Number of Time Cycles per Year':
            cycles = line.split('\t')[1].strip()            
        
        if line.split('\t')[0].strip() == 'Maximum Number of Treatment Units':
            maxTU = line.split('\t')[1].strip()            

        if line.split('\t')[0].strip() == 'Maximum Number of Pipes':
            maxPipes = line.split('\t')[1].strip()            

        if line.split('\t')[0].strip() == 'Maximum Pipe Capacity':
            maxPipeCap = line.split('\t')[1].strip()            

        if line.split('\t')[0].strip() == 'Number of Pipe Capacity Intervals':
            PipeCapInt = line.split('\t')[1].strip()            

        if line.split('\t')[0].strip() == 'Local Recycle Allowed':
            recycle = line.split('\t')[1].strip()            

        if line.split('\t')[0].strip() == 'Steady State':
            steady = line.split('\t')[1].strip()
            

        if line.split('\t')[0].strip().startswith('WS'):
            sourcetimecost[line.split('\t')[0].strip()][line.split('\t')[1].strip()] = line.split('\t')[2].strip()
            sourcetimeminflow[line.split('\t')[0].strip()][line.split('\t')[1].strip()] = line.split('\t')[3].strip()
            sourcetimemaxflow[line.split('\t')[0].strip()][line.split('\t')[1].strip()] = line.split('\t')[4].strip()
            for i in range(5,5+int(len(contaminants))):
                sourcetimecontconc[line.split('\t')[0].strip()][line.split('\t')[1].strip()][contaminants[i-5]] = line.split('\t')[i].strip()
            
        if line.split('\t')[0].strip().startswith('PU'):
            PUminFlow[line.split('\t')[0].strip()] = line.split('\t')[1].strip()
            PUmaxFlow[line.split('\t')[0].strip()] = line.split('\t')[2].strip()
            PUbalance[line.split('\t')[0].strip()] = line.split('\t')[3].strip()
            PUstart[line.split('\t')[0].strip()] = line.split('\t')[4].strip()
            PUduration[line.split('\t')[0].strip()] = line.split('\t')[5].strip()
            PUend[line.split('\t')[0].strip()] = line.split('\t')[6].strip()
            for i in range(7,7+int(len(contaminants))):
                PUcontdisc[line.split('\t')[0].strip()][contaminants[i-7]] = line.split('\t')[i].strip()
            for i in range(7+int(len(contaminants)),7+2*int(len(contaminants))):
                PUcontin[line.split('\t')[0].strip()][contaminants[i-7-len(contaminants)]] = line.split('\t')[i].strip()
            for i in range(7+2*int(len(contaminants)),7+3*int(len(contaminants))):
                PUcontout[line.split('\t')[0].strip()][contaminants[i-7-2*len(contaminants)]] = line.split('\t')[i].strip()

        if line.split('\t')[0].strip().startswith('TU'):
            TUtype[line.split('\t')[0].strip()] = line.split('\t')[1].strip()
            TUinvCost[line.split('\t')[0].strip()] = line.split('\t')[2].strip()
            TUopCost[line.split('\t')[0].strip()] = line.split('\t')[3].strip()
            TUminFlow[line.split('\t')[0].strip()] = line.split('\t')[4].strip()
            TUmaxFlow[line.split('\t')[0].strip()] = line.split('\t')[5].strip()
            TUduration[line.split('\t')[0].strip()] = line.split('\t')[6].strip()
            for i in range(7,7+int(len(contaminants))):
                TUconttypechar[line.split('\t')[0].strip()][contaminants[i-7]] = line.split('\t')[i].strip()
            for i in range(7+int(len(contaminants)),7+2*int(len(contaminants))):
                TUcontin[line.split('\t')[0].strip()][contaminants[i-7-len(contaminants)]] = line.split('\t')[i].strip()
                        
        if line.split('\t')[0].strip().startswith('WD'):
            desttimeminflow[line.split('\t')[0].strip()][line.split('\t')[1].strip()] = line.split('\t')[2].strip()
            desttimemaxflow[line.split('\t')[0].strip()][line.split('\t')[1].strip()] = line.split('\t')[3].strip()
            for i in range(4,4+int(len(contaminants))):
                desttimecontconc[line.split('\t')[0].strip()][line.split('\t')[1].strip()][contaminants[i-4]] = line.split('\t')[i].strip()

    f.close()
    
    
    
    # assertions that check whether all is as easy as we assume (steady state)
    assert(int(steady) == 1)

    assert(len(buffers) == 0)
    
    assert(int(PipeCapInt) < 1000)
    
    for p in processunits:
        assert(int(PUduration[p]) == 0)
        # this is for later...
        if int(PUduration[p]) > 0:
            assert(PUstart[p] + PUduration[p] == PUend[p])
            
    for t in treatmentunits:
        assert(int(TUduration[t]) == 0)
    
    fout.write('Set\tNetElements\t/\n')
    for item in watersources:
        fout.write('\t'+item+"\n")
    for item in processunits:
        fout.write('\t'+item+"\n")
    for item in treatmentunits:
        fout.write('\t'+item+"\n")
    for item in destinations:
        fout.write('\t'+item+"\n")
    fout.write('/;\n\n')
    
    fout.write('Sets\tWS(NetElements)\t/\n')
    for item in watersources:
        fout.write('\t'+item+"\n")
    fout.write('\t/\n')
    fout.write('\tPU(NetElements)\t/\n')
    for item in processunits:
        fout.write('\t'+item+"\n")
    fout.write('\t/\n')
    fout.write('\tTU(NetElements)\t/\n')
    for item in treatmentunits:
        fout.write('\t'+item+"\n")
    fout.write('\t/\n')
    fout.write('\tWD(NetElements)\t/\n')
    for item in destinations:
        fout.write('\t'+item+"\n")
    fout.write('\t/\n')
    fout.write('\tT\t/\n')
    for item in times:
        fout.write('\t'+item+"\n")
    fout.write('\t/\n')
    fout.write('\tC\t/\n')
    for item in contaminants:
        fout.write('\t'+item+"\n")
    fout.write('/;\n\n')
    
    fout.write('Sets\tSplitters(NetElements)\n\tMixers(NetElements)\n;\n\n')
    fout.write('Splitters(NetElements) = WS(NetElements) + PU(NetElements) + TU(NetElements);\n')
    fout.write('Mixers(NetElements) = WD(NetElements) + PU(NetElements) + TU(NetElements);\n\n')
    
    fout.write('Scalars')
    fout.write('\tcycles / '+cycles+' /\n')
    fout.write('\tmaxTU / '+maxTU+' /\n')
    fout.write('\tmaxPipes / '+maxPipes+' /\n')
    fout.write('\tmaxPipeCap / '+maxPipeCap+' /\n')
    fout.write('\tPipeCapInt / '+PipeCapInt+' /\n')
    fout.write('\trecycle / '+recycle+' /\n')
    fout.write('\tsteady / '+steady+' /\n')
    fout.write(';\n\n')
    
    fout.write('Parameter WaterCost(WS,T) /')
    for s in watersources:
        for t in times:
            fout.write('\n\t'+s+' . '+t+'\t = '+sourcetimecost[s][t])
    fout.write(' /;\n\n')
    
    fout.write('Parameter SourceMinFlow(WS,T) /')
    for s in watersources:
        for t in times:
            fout.write('\n\t'+s+' . '+t+'\t'+sourcetimeminflow[s][t])
    fout.write(' /;\n\n')
    
    fout.write('Parameter SourceMaxFlow(WS,T) /')
    for s in watersources:
        for t in times:
            fout.write('\n\t'+s+' . '+t+'\t'+sourcetimemaxflow[s][t])
    fout.write(' /;\n\n')
    
    fout.write('Parameter SourceCont(WS,T,C) /')
    for s in watersources:
        for t in times:
            for c in contaminants:
                fout.write('\n\t'+s+' . '+t+' . '+c+'\t'+sourcetimecontconc[s][t][c])
    fout.write(' /;\n\n')
    
    fout.write('Parameter PUminFlow(PU) /')
    for p in processunits:
        fout.write('\n\t'+p+'\t'+PUminFlow[p])
    fout.write(' /;\n\n')
    
    fout.write('Parameter PUmaxFlow(PU) /')
    for p in processunits:
        fout.write('\n\t'+p+'\t'+PUmaxFlow[p])
    fout.write(' /;\n\n')

    fout.write('Parameter PUbalance(PU) /')
    for p in processunits:
        fout.write('\n\t'+p+'\t'+PUbalance[p])
    fout.write(' /;\n\n')

    fout.write('Parameter PUstart(PU) /')
    for p in processunits:
        fout.write('\n\t'+p+'\t'+PUstart[p])
    fout.write(' /;\n\n')

    fout.write('Parameter PUduration(PU) /')
    for p in processunits:
        fout.write('\n\t'+p+'\t'+PUduration[p])
    fout.write(' /;\n\n')
    
    fout.write('Parameter PUend(PU) /')
    for p in processunits:
        fout.write('\n\t'+p+'\t'+PUend[p])
    fout.write(' /;\n\n')
    
    fout.write('Parameter ContDischarge(PU,C) /')
    for p in processunits:
        for c in contaminants:
            fout.write('\n\t'+p+' . '+c+'\t'+PUcontdisc[p][c])
    fout.write(' /;\n\n')

    fout.write('Parameter PUcontInflow(PU,C) /')
    for p in processunits:
        for c in contaminants:
            fout.write('\n\t'+p+' . '+c+'\t'+PUcontin[p][c])
    fout.write(' /;\n\n')

    fout.write('Parameter PUcontOutflow(PU,C) /')
    for p in processunits:
        for c in contaminants:
            fout.write('\n\t'+p+' . '+c+'\t'+PUcontout[p][c])
    fout.write(' /;\n\n')

    fout.write('Parameter TUtype(TU) /')
    for t in treatmentunits:
        fout.write('\n\t'+t+'\t'+TUtype[t])
    fout.write(' /;\n\n')
    
    fout.write('Parameter TUinvcost(TU) /')
    for t in treatmentunits:
        fout.write('\n\t'+t+'\t'+TUinvCost[t])
    fout.write(' /;\n\n')

    fout.write('Parameter TUopCost(TU) /')
    for t in treatmentunits:
        fout.write('\n\t'+t+'\t'+TUopCost[t])
    fout.write(' /;\n\n')

    fout.write('Parameter TUminFlow(TU) /')
    for t in treatmentunits:
        fout.write('\n\t'+t+'\t'+TUminFlow[t])
    fout.write(' /;\n\n')

    fout.write('Parameter TUmaxFlow(TU) /')
    for t in treatmentunits:
        fout.write('\n\t'+t+'\t'+TUmaxFlow[t])
    fout.write(' /;\n\n')

    fout.write('Parameter TUduration(TU) /')
    for t in treatmentunits:
        fout.write('\n\t'+t+'\t'+TUduration[t])
    fout.write(' /;\n\n')

    fout.write('Parameter TUcontTypeChar(TU,C) /')
    for t in treatmentunits:
        for c in contaminants:
            fout.write('\n\t'+t+' . '+c+'\t'+TUconttypechar[t][c])
    fout.write(' /;\n\n')

    fout.write('Parameter TUcontin(TU,C) /')
    for t in treatmentunits:
        for c in contaminants:
            fout.write('\n\t'+t+' . '+c+'\t'+TUcontin[t][c])
    fout.write(' /;\n\n')

    fout.write('Parameter DestMinFlow(WD,T) /')
    for s in destinations:
        for t in times:
            fout.write('\n\t'+s+' . '+t+'\t'+desttimeminflow[s][t])
    fout.write(' /;\n\n')
    
    fout.write('Parameter DestMaxFlow(WD,T) /')
    for s in destinations:
        for t in times:
            fout.write('\n\t'+s+' . '+t+'\t'+desttimemaxflow[s][t])
    fout.write(' /;\n\n')
    
    fout.write('Parameter DestCont(WD,T,C) /')
    for s in destinations:
        for t in times:
            for c in contaminants:
                fout.write('\n\t'+s+' . '+t+' . '+c+'\t'+desttimecontconc[s][t][c])
    fout.write(' /;\n\n')
    
    fout.close()